In [2]:
from __future__ import print_function
from pathlib import Path
from random import shuffle
import random

import argparse
import copy
import numpy as np
import sys
import tensorflow as tf
import scipy.sparse as sp

from deeplp.models.data_prep import create_weighted_graph, load_data
from deeplp.models.data_prep import prepare_data, random_unlabel, calc_masks
from deeplp.models.deeplp_att import DeepLP_ATT
from deeplp.models.deeplp_edge import DeepLP_Edge
from deeplp.models.deeplp_wrbf import DeepLP_WRBF
from deeplp.models.lp import LP
from deeplp.models.utils import accuracy, indices_to_vec
from deeplp.models.data_prep import select_features


/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
def approx_chunk(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [15]:
t=[]
for p in [0.99,0.98,0.97,0.96,0.95,0.94,0.93,0.92,0.91,0.9]:
    accuracies = np.zeros((1,10,10))
    for i,data in enumerate(['cora']):

        for seed in range(10):
            true_labels, edge_features, node_features, graph \
            = load_data(data,'linqs',directed=1,confidence=0)

            labeled_indices, unlabeled_indices = \
                random_unlabel(true_labels,p,
                               seed=seed,confidence=0)
            num_nodes, num_classes = true_labels.shape

            labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)

            for j in range(1,11):
#                 print(data,seed,j)
                final_accs = []

                labeled_indices_copy = copy.copy(labeled_indices)
                random.seed(seed)
                shuffle(labeled_indices_copy)
                cv_held_out_indices_list = approx_chunk(labeled_indices_copy, 5)


                for k, cv_held_out_indices in enumerate(cv_held_out_indices_list):

                    cv_labeled_indices = [index for index in labeled_indices if index not in cv_held_out_indices]
                    cv_unlabeled_indices = np.delete(np.arange(true_labels.shape[0]),cv_labeled_indices)
                    cv_labels, cv_is_labeled = calc_masks(true_labels, cv_labeled_indices, cv_unlabeled_indices,logistic=0)
                    lp = LP()
                    index = np.hstack([cv_held_out_indices,cv_unlabeled_indices])
                    unlabeled_pred = lp.iter_sp(cv_labels,
                                         graph,
                                         cv_is_labeled,
                                         10 * j,
                                         index)

                    y_pred = np.argmax(unlabeled_pred,axis=1)
                    y_true = np.argmax(true_labels[index],axis=1)
                    acc = np.mean(y_pred == y_true)
                    final_accs.append(acc)
                acc = np.mean(final_accs)
                accuracies[i,seed,j-1] = acc
#                 print("baseline",acc)
    t.append(np.argmax(np.mean(accuracies,axis=1)[0]))

-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!


(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric


In [16]:
t

[1, 2, 2, 2, 2, 3, 3, 3, 3, 2]

In [10]:
for i in np.mean(accuracies,axis=1)[0]:
    print(i)

0.740846473029
0.753858921162
0.755692946058
0.754315352697
0.752190871369
0.749850622407
0.747900414938
0.745784232365
0.744282157676
0.743136929461


In [11]:
np.argmax(np.mean(accuracies,axis=1)[0])

2

In [3]:
fin_accs = np.zeros((5,10))
i=0

# for data,num_iter in zip(['flickr'],[10]):
for data,num_iter in zip(['flickr'],[10]):


# for data,num_iter in zip(['citeseer','cora','pubmed'],[20,50,20]):

    for seed in range(10):
        true_labels, features, edge_features, node_features, graph \
        = load_data(data,'flip',directed=1,confidence=0)

        labeled_indices, unlabeled_indices = \
            random_unlabel(true_labels,0.99,features,
                           seed=seed,confidence=0)
        num_nodes, num_classes = true_labels.shape

        labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)

        lp = LP()
        unlabeled_pred = lp.iter_sp(labels,
                             graph,
                             is_labeled,
                             num_iter,
                             unlabeled_indices)

        y_pred = np.argmax(unlabeled_pred,axis=1)
        y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
        acc = np.mean(y_pred == y_true)
        print(acc)
        fin_accs[i,seed] = acc
    i+=1

-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.6352806995311114
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.7442656190596882
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.5529083766316056
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.547712583956406
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.7225953618045875
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.7175262957800025
-----------flickr-----------
Loading labels...
Loading features...
Loading edge features...
Loading graph...
Done!
0.5886452921049297
-----------flickr-----------
Loading labels...
Loading features

In [4]:
np.mean(fin_accs,axis=1)

array([0.64403751, 0.        , 0.        , 0.        , 0.        ])

In [9]:
t=np.zeros((6,5))
for i,p in enumerate([0.99,0.98,0.96,0.94,0.92,0.9]):
    for seed in range(5):
        true_labels, edge_features, node_features, graph \
        = load_data('cora','linqs',directed=1,confidence=0)

        labeled_indices, unlabeled_indices = \
            random_unlabel(true_labels,p,
                           seed=seed,confidence=0)
        num_nodes, num_classes = true_labels.shape

        labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)

        lp = LP()
        unlabeled_pred = lp.iter_sp(labels,
                             graph,
                             is_labeled,
                             30,
                             unlabeled_indices)

        y_pred = np.argmax(unlabeled_pred,axis=1)
        y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
        acc = np.mean(y_pred == y_true)
        t[i,seed] = acc


-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading edge features...
Weights: Asymmetric
(10138, 20)
Loading graph...
Done!


In [11]:
np.mean(t,1)

array([ 0.66813008,  0.70546201,  0.75513627,  0.79014989,  0.80279965,
        0.80724508])

In [12]:
t

array([[ 0.68455285,  0.64593496,  0.68943089,  0.6304878 ,  0.6902439 ],
       [ 0.71868583,  0.69117043,  0.68172485,  0.69979466,  0.73593429],
       [ 0.76519916,  0.76310273,  0.75220126,  0.74591195,  0.74926625],
       [ 0.79571734,  0.79057816,  0.81670236,  0.77216274,  0.77558887],
       [ 0.81014873,  0.7895888 ,  0.82983377,  0.80271216,  0.78171479],
       [ 0.81797853,  0.78890877,  0.8206619 ,  0.80858676,  0.80008945]])